## Data Access

In [0]:
secret = dbutils.secrets.get(scope="databricks-key12", key="secret-id")
app_id = dbutils.secrets.get(scope="databricks-key12",key="app-id")
dir_id = dbutils.secrets.get(scope="databricks-key12", key="dir-id")

In [0]:
#service_credential = dbutils.secrets.get(scope="<secret-scope>",key="<service-credential-key>")

spark.conf.set("fs.azure.account.auth.type.nyctaxistorageabhinav.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.nyctaxistorageabhinav.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.nyctaxistorageabhinav.dfs.core.windows.net", app_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.nyctaxistorageabhinav.dfs.core.windows.net", secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.nyctaxistorageabhinav.dfs.core.windows.net", f"https://login.microsoftonline.com/{dir_id}/oauth2/token")

# Database Creation

In [0]:
%sql
--CREATE DATABASE gold

In [0]:
%sql
USE gold;

In [0]:
%sql
--SHOW STORAGE CREDENTIALS;

In [0]:
%sql
--CREATE EXTERNAL LOCATION gold_root
--URL 'abfss://gold@nyctaxistorageabhinav.dfs.core.windows.net/'
--WITH (STORAGE CREDENTIAL gold_azuremanagedidentity_1761769381602)
--COMMENT 'Gold layer root external location';

In [0]:
%sql
show EXTERNAL LOCATIONS;

In [0]:
%sql
--GRANT READ FILES, WRITE FILES ON EXTERNAL LOCATION gold_root TO `buddybanner1234@outlook.com`;

In [0]:
%sql
--SELECT current_user();

In [0]:
%sql
--SHOW EXTERNAL LOCATIONS;

In [0]:
dbutils.fs.ls("abfss://gold@nyctaxistorageabhinav.dfs.core.windows.net/")


CREATE STORAGE CREDENTIAL nyctaxi_cred
WITH AZURE_SERVICE_PRINCIPAL (
  CLIENT_ID '<app_id>',
  CLIENT_SECRET '<secret>',
  TENANT_ID '<dir_id>'
);

# Data Reading and Writing and Creating delta tables

In [0]:
  from pyspark.sql.functions import *
  from pyspark.sql.types import *

### Storage Variables

In [0]:
silver = "abfss://silver@nyctaxistorageabhinav.dfs.core.windows.net/"
gold = "abfss://gold@nyctaxistorageabhinav.dfs.core.windows.net/"

**Data zone**

###### Trip Zone

In [0]:
df_zone = spark.read.format("parquet")\
  .option("InferSchema", "true")\
  .option("header", "true")\
  .load(f"{silver}/trip_zone")

display(df_zone.limit(5))

In [0]:
df_zone.write.format("delta")\
    .mode("append")\
    .option("path", f"{gold}trip_zone")\
    .saveAsTable("gold.trip_zone")

### Trip 

In [0]:
df_trip_type = spark.read.format("parquet")\
  .option("InferSchema", "true")\
  .option("header", "true")\
  .load(f"{silver}/trip_type")

display(df_trip_type.limit(5))

In [0]:
df_trip_type.write.format("delta")\
    .mode("append")\
    .option("path", f"{gold}trip_type")\
    .saveAsTable("gold.trip_type")

In [0]:
df_trip = spark.read.format("parquet")\
  .option("InferSchema", "true")\
  .option("header", "true")\
    .option("recursiveFileLookup", "true")\
  .load(f"{silver}trip/trips2024")

display(df_trip.limit(5))

In [0]:
df_trip.write.format("delta")\
    .mode("append")\
    .option("path", f"{gold}trip")\
    .saveAsTable("gold.trip")

In [0]:
df_zone.write.format("delta") \
    .mode("overwrite") \
    .option("path", "abfss://gold@nyctaxistorageabhinav.dfs.core.windows.net/trip_zone") \
    .saveAsTable("gold.trip_zone")

# Learning

In [0]:
%sql
select * from gold.trip_zone;

In [0]:
%sql
UPDATE gold.trip_zone
SET Borough = "EMR"
WHERE LocationID=1;

In [0]:
%sql
delete from gold.trip_zone
where LocationID=1;

In [0]:
%sql
describe history gold.trip_zone;

In [0]:
%sql
-- time travel
restore gold.trip_zone to version as of 0;
    
-- time travel

In [0]:
%sql
select * from gold.trip_zone;

# Delta Tables

#### Trip Zone

In [0]:
%sql
select * from gold.trip_zone

#### Trip Type

In [0]:
%sql
select * from gold.trip_type;

#### Trip Data 2024

In [0]:
%sql
select * from gold.trip;

In [0]:
%sql
--drop table df_trip_type;